### Eksperymenty

**Modele**
- resnet50
- vgg16

**Zbiory danych**
- CIFAR100
- SVHN

**Metody**
- Random
- KMeans
- KMeansPurity
- KMeansDino
- KMeansDinoPurity

**Ustawienia**
- ratio = (0.1, 1, 10)
- epochs = 10
- batch_size = 512
- clip = 5.0
- num_clusters = 8
- eqsize = True
- min_purity = 0.05 (SVHN), 0.01 (CIFAR100)

**Nazewnictwo przebiegów**
W nawiasach kwadratowych podano wartość do wyboru ("jeden z ...") w nawiasch klamrowych wartości opcjonalne.
ID służy do rozróżniania eksperymentów w tej samej konfiguracji

- [r50,vgg13],ep:10,bs:512,clip:5.0,{nc:20},{eqsize},{mp:[0.1,0.01]},ID:[1,2,3,..]
- przykład: r50,ep:10,bs:512,clip:5.0,ID:1
- przykład: r50,ep:10,bs:512,clip:50,nc:20,eqsize,mp:0.1,ID:1

**Uwagi**
- Przy każdym eksperymencie nowy model (!)

In [1]:
import torch
import torchvision
from torchvision.models import resnet50

from kiss.models import vgg16_kiss
from kiss.experiment import Experiment
from kiss.sampler import RandomSampler, KMeansSampler, KMeansPuritySampler, KMeansDinoSampler, KMeansPurityDinoSampler
from kiss.utils.configs import CONFIGS

transform = torchvision.transforms.ToTensor()
dataset_cifar100_tr = torchvision.datasets.CIFAR100(root='../data/cifar100', train=True, download=True, transform=transform)
dataset_cifar100_te = torchvision.datasets.CIFAR100(root='../data/cifar100', train=False, download=True, transform=transform)

dataset_svhn_tr = torchvision.datasets.SVHN(root='../data/svhn', split='train', download=True, transform=transform)
dataset_svhn_te = torchvision.datasets.SVHN(root='../data/svhn', split='test', download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ../data/svhn/train_32x32.mat
Using downloaded and verified file: ../data/svhn/test_32x32.mat


In [2]:
ID = 2
RATIO = (0.1, 1, 10)
EPOCHS = 10
BATCH_SIZE = 512
CLIP = 5.0
NUM_CLUSTERS = 8
EQSIZE = True

In [3]:
import os
    
for model_fun in [vgg16_kiss]:
    for dataset_tr, dataset_te in zip([dataset_cifar100_tr, dataset_svhn_tr], [dataset_cifar100_te, dataset_svhn_te]):
        for sampler_cls in [RandomSampler, KMeansSampler, KMeansPuritySampler]:

            if model_fun.__name__ == 'resnet50':
                MODEL_NAME = 'r50'
            if model_fun.__name__ == 'vgg16':
                MODEL_NAME = 'vgg16'
            if model_fun.__name__ == 'vgg16_kiss':
                MODEL_NAME = 'vgg16_kiss'

            RUN_NAME = f"{MODEL_NAME},ep:{EPOCHS},bs:{BATCH_SIZE},clip:{CLIP}"

            if dataset_tr.__class__.__name__ == 'SVHN':
                NUM_CLASSES = 10
                MIN_PURITY = 0.05
            if dataset_tr.__class__.__name__ == 'CIFAR100':
                NUM_CLASSES = 100
                MIN_PURITY = 0.01

            if 'KMeans' in sampler_cls.__name__:
                RUN_NAME += f",nc:{NUM_CLUSTERS}"

            if 'KMeans' in sampler_cls.__name__ and EQSIZE:
                RUN_NAME += ",eqsize"

            if 'KMeansPurity' in sampler_cls.__name__:
                RUN_NAME += f",mp:{MIN_PURITY}"

            SAVE_CLUSTERS = LOAD_CLUSTERS = f"../checkpoints/{sampler_cls.__name__},{dataset_tr.__class__.__name__},nc:{NUM_CLUSTERS}"
            
            if not os.path.exists(LOAD_CLUSTERS):
                LOAD_CLUSTERS = None
                
            RUN_NAME += f',ID:{ID}'

            model = model_fun(num_classes=NUM_CLASSES)
            model.to(torch.device(CONFIGS.torch.device))

            experiment = Experiment(
                model = model,
                dataset_tr = dataset_tr,
                dataset_te = dataset_te,
                sampler_cls=sampler_cls,
                ratio=RATIO,
                epochs=EPOCHS,
                batch_size=BATCH_SIZE,
                clip=CLIP,
                num_clusters=NUM_CLUSTERS,
                eqsize=EQSIZE,
                min_purity=MIN_PURITY,
                load_clusters=LOAD_CLUSTERS,
                save_clusters=SAVE_CLUSTERS,
            )
            experiment.run("../experiments", RUN_NAME)

Running experiment VGG!CIFAR100!RandomSampler
Running run vgg16_kiss,ep:10,bs:512,clip:5.0,ID:2/1


Validating: 100%|██████████| 2/2 [00:00<00:00,  9.52 batch/s, loss=4.6079]


Best valid loss improved. Current accuracy is 0.80%. Saving checkpoint...
Best valid accuracy improved. Current accuracy is 0.80%. Saving checkpoint...


Validating: 100%|██████████| 2/2 [00:00<00:00, 10.33 batch/s, loss=4.5778]


Best valid loss improved. Current accuracy is 1.40%. Saving checkpoint...
Best valid accuracy improved. Current accuracy is 1.40%. Saving checkpoint...


...

Validating: 100%|██████████| 6/6 [00:00<00:00, 11.31 batch/s, loss=2.2599]


Best valid loss improved. Current accuracy is 18.49%. Saving checkpoint...
Best valid accuracy improved. Current accuracy is 18.49%. Saving checkpoint...


Validating: 100%|██████████| 6/6 [00:00<00:00, 10.42 batch/s, loss=1.9364]


Best valid loss improved. Current accuracy is 25.73%. Saving checkpoint...
Best valid accuracy improved. Current accuracy is 25.73%. Saving checkpoint...


Epoch 4/10:  96%|█████████▌| 22/23 [00:06<00:00,  3.29 batch/s, loss=1.7271]

In [ ]:
RUN_NAME = f"r50,ep:10,bs:512,clip:5.0,ID:{ID}"

model = resnet50(num_classes=100)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_cifar100_tr,
    dataset_cifar100_te, 
    RandomSampler,
    ratio=RATIO,
    epochs=10,
    batch_size=512,
    clip=5.0)
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"r50,ep:10,bs:512,clip:5.0,ID:{ID}"

model = resnet50(num_classes=10)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_svhn_tr,
    dataset_svhn_te, 
    RandomSampler,
    ratio=RATIO,
    epochs=10,
    batch_size=512,
    clip=5.0)
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"r50,ep:10,bs:512,clip:5.0,nc:8,eqsize,ID:{ID}"

model = resnet50(num_classes=100)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_cifar100_tr,
    dataset_cifar100_te, 
    KMeansSampler,
    ratio=RATIO,
    epochs=10,
    batch_size=512,
    clip=5.0,
    num_clusters=8,
    eqsize=True,
    save_clusters="../checkpoints/kmeans,cifar100")
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"r50,ep:10,bs:512,clip:5.0,nc:8,eqsize,ID:{ID}"

model = resnet50(num_classes=10)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_svhn_tr,
    dataset_svhn_te, 
    KMeansSampler,
    ratio=RATIO,
    epochs=10,
    batch_size=512,
    clip=5.0,
    num_clusters=8,
    eqsize=True,
    save_clusters="../checkpoints/kmeans,svhn")
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"r50,ep:10,bs:512,clip:5.0,nc:8,eqsize,mp:0.01,ID:{ID}"

model = resnet50(num_classes=100)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_cifar100_tr,
    dataset_cifar100_te, 
    KMeansPuritySampler,
    ratio=RATIO,
    epochs=10,
    batch_size=512,
    clip=5.0,
    num_clusters=8,
    min_purity=0.01,
    eqsize=True,
    save_clusters="../checkpoints/kmeans-purity,cifar100")
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"r50,ep:10,bs:512,clip:5.0,nc:8,eqsize,mp:0.05,ID:{ID}"

model = resnet50(num_classes=10)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_svhn_tr,
    dataset_svhn_te, 
    KMeansPuritySampler,
    ratio=RATIO,
    epochs=10,
    batch_size=512,
    clip=5.0,
    num_clusters=8,
    min_purity=0.05,
    eqsize=True,
    save_clusters="../checkpoints/kmeans-purity,svhn")
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"vgg13,ep:10,bs:512,clip:5.0,ID:{ID}"

model = vgg13(num_classes=100)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_cifar100_tr,
    dataset_cifar100_te, 
    RandomSampler,
    ratio=RATIO,
    epochs=10,
    batch_size=512,
    clip=5.0)
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"vgg13,ep:10,bs:512,clip:5.0,ID:{ID}"

model = vgg13(num_classes=10)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_svhn_tr,
    dataset_svhn_te, 
    RandomSampler,
    ratio=RATIO,
    epochs=10,
    batch_size=512,
    clip=5.0)
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"vgg13,ep:10,bs:512,clip:5.0,nc:8,eqsize,ID:{ID}"

model = vgg13(num_classes=100)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_cifar100_tr,
    dataset_cifar100_te, 
    KMeansSampler,
    ratio=RATIO,
    epochs=10,
    batch_size=512,
    clip=5.0,
    num_clusters=8,
    eqsize=True,
    load_clusters="../checkpoints/kmeans,cifar100")
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"vgg13,ep:10,bs:512,clip:5.0,nc:8,eqsize,ID:{ID}"

model = vgg13(num_classes=10)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_svhn_tr,
    dataset_svhn_te, 
    KMeansSampler,
    ratio=RATIO,
    epochs=10,
    batch_size=512,
    clip=5.0,
    num_clusters=8,
    eqsize=True,
    load_clusters="../checkpoints/kmeans,svhn")
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"vgg13,ep:10,bs:512,clip:5.0,nc:8,eqsize,mp:0.1,ID:{ID}"

model = vgg13(num_classes=100)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_cifar100_tr,
    dataset_cifar100_te, 
    KMeansPuritySampler,
    ratio=RATIO,
    epochs=10,
    batch_size=512,
    clip=5.0,
    num_clusters=8,
    min_purity=0.1,
    eqsize=True,
    load_clusters="../checkpoints/kmeans-purity,cifar100")
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"vgg13,ep:10,bs:512,clip:5.0,nc:8,eqsize,mp:0.05,ID:{ID}"

model = vgg13(num_classes=10)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_svhn_tr,
    dataset_svhn_te, 
    KMeansPuritySampler,
    ratio=RATIO,
    epochs=10,
    batch_size=512,
    clip=5.0,
    num_clusters=8,
    min_purity=0.05,
    eqsize=True,
    load_clusters="../checkpoints/kmeans-purity,svhn")
    
experiment.run("../experiments", RUN_NAME)